In [9]:
import sys
sys.path.append('/Users/chemlab/Google Drive/My Drive/Data/GoodSubstrateFiles')
sys.path.append("/Users/chemlab/Documents/SubstituentCoding/QTAIM Reorientation Workspace/scripts")
sys.path.append(sys.path[0].replace('Development Notebooks',''))
from subproptools import qtaimExtract as qt
from subproptools import subreor as sr
import pandas as pd
import numpy as np
import math


In [10]:
def get_all_bcp_pairs(data):
    atPairs = []
    for line in data:
        if '(3,-1)' in line:
            splitLine = line.split()
            atPairs.append([splitLine[4],splitLine[5]])
    return atPairs        

In [11]:
'Type = (3,-1) BCP S1 Si2'.split()


['Type', '=', '(3,-1)', 'BCP', 'S1', 'Si2']

In [12]:
sumFile = open('/Users/chemlab/Google Drive/My Drive/Data/GoodSubstrateFiles/'+'SubGeH3_CCl3-ReorJune-B3LYP-def2-TZVPPD-Field'+".sum","r") #open file, read lines, close file
data = sumFile.readlines()
sumFile.close()

In [13]:
def get_all_bcp_properties(sumfile,pathToSum='/Users/chemlab/Google Drive/My Drive/Data/GoodSubstrateFiles/'):
    sumFileOpen = open(pathToSum+sumfile+".sum","r") #open file, read lines, close file
    data = sumFileOpen.readlines()
    sumFileOpen.close()
    bcpPairList = get_all_bcp_pairs(data)
    isFirst=0
    for pair in bcpPairList:
        block = qt.get_bcp_block(data,pair)
        properties = qt.get_bcp_properties(block)

        diTable = qt.get_di_table(data)
        for ind in diTable.index:
            if (diTable['Atom A'][ind] == pair[0] and diTable['Atom B'][ind] == pair[1]) or (diTable['Atom A'][ind] == pair[1] and diTable['Atom B'][ind] == pair[0]):
                properties.update({'DI': diTable['DI(A,B)'][ind]})
                break
        del properties['xyz']
        if isFirst == 0:
            isFirst = 1
            bcpFrame = pd.DataFrame.from_dict(properties,orient='columns')
        else:
            bcpFrame = pd.concat([bcpFrame,pd.DataFrame(properties)],ignore_index=True)
    return bcpFrame                        

In [14]:
get_all_bcp_properties('SubGeH3_CCl3-ReorJune-B3LYP-def2-TZVPPD-Field')

,rho,lambda1,lambda2,lambda3,DelSqRho,Ellipticity,V,G,H,DI
0,0.116537,-0.143375,-0.143372,0.403487,0.116740,0.000017,-0.136510,0.082848,-0.053663,6.5917708734E-01
1,0.185585,-0.294724,-0.280871,0.351361,-0.224234,0.049321,-0.187132,0.065537,-0.121595,1.0845456782E+00
2,0.185586,-0.294729,-0.280872,0.351363,-0.224238,0.049337,-0.187132,0.065536,-0.121596,1.0844135600E+00
3,0.185586,-0.294729,-0.280872,0.351363,-0.224238,0.049337,-0.187132,0.065536,-0.121596,1.0844959447E+00
4,0.136078,-0.207244,-0.205598,0.477021,0.064179,0.008007,-0.171946,0.093995,-0.077950,8.2505969563E-01
5,0.136078,-0.207244,-0.205598,0.477021,0.064179,0.008007,-0.171946,0.093995,-0.077950,8.2510382169E-01
6,0.136076,-0.207241,-0.205595,0.477020,0.064184,0.008005,-0.171944,0.093995,-0.077949,8.2501099068E-01


In [15]:
def get_props_multifile(csvFile):
    csvFrame = pd.read_csv(csvFile)
    nrow = csvFrame.shape[0]
    for drivesumFile in range(0,nrow):
        print(csvFrame.loc[drivesumFile]['sumFileToGet'])
        bcpProperties = get_all_bcp_properties(csvFrame.loc[drivesumFile]['sumFileToGet'])
        if drivesumFile==0:
            outFrame = bcpProperties
        else:
            outFrame = pd.concat([outFrame,bcpProperties])    
    return outFrame        

In [16]:
bcpPropertyFrame = get_props_multifile('sumfilesToExtract.csv')

SubBCP_AlH2-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_BeH-Reor2-B3LYP-def2-TZVPPDTRUE-Field
SubBCP_BH2-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_C6H5-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CCCl-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CCF-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_C.CH3.3-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CCH-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CCl2F-Reor2020-B3LYP-def2-TZVPPD-Field
SubBCP_CCl2H-Reor-B3LYP-def2-TZVPPD-Field
SubBCP_CCl3-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CClF2-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CClFH-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CClH2-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CF2H-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CF3-ReorJune-B3LYP-def2-TZVPPD-Field
SubBCP_CFH2-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CH2CH2CH3-IP-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CH2CH2CH3-OoP-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CH2CH3-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CH2CH-CH3-2-IP-ReorPosY-B3LYP-def2-TZVPPD-Field
SubBCP_CH2CH-CH3-2-OoP-Reo

In [20]:
bcpPropertyFrame['DI'] = bcpPropertyFrame['DI'].astype(float)

In [21]:
bcpPropertyFrame.std(axis=0)

rho            0.077290
lambda1        0.299756
lambda2        0.291123
lambda3        0.449474
DelSqRho       0.570553
Ellipticity    0.137890
V              0.332327
G              0.184923
H              0.176627
DI             0.369324
dtype: float64

In [22]:
bcpPropertyFrame.mean(axis=0)

rho            0.290686
lambda1       -0.725552
lambda2       -0.678830
lambda3        0.583261
DelSqRho      -0.821120
Ellipticity    0.077722
V             -0.475963
G              0.135342
H             -0.340622
DI             1.081894
dtype: float64